# Example notebook

Here we get the Article classes for all given bio.tools tools.

In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
from bh24_literature_mining.utils import load_biotools_pub
from bh24_literature_mining.biotools import Tool_entry

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

# Initialize set to track unique names and a list to store Biotool objects
tools_lower = set()
unique_biotools = []

# Iterate over the rows in the DataFrame
for _, row in tools.iterrows():
    name = row["name"]
    biotools_id = row["biotoolsID"]
    name_lower = name.lower()

    # Only add unique tools based on name
    if name_lower not in tools_lower:
        tools_lower.add(name_lower)  # Add to the set to track uniqueness
        # Create a Biotool object and add it to the list
        biotool = Tool_entry(biotool_id=biotools_id, name=name)
        unique_biotools.append(biotool)


print(len(unique_biotools))

15005


## Example on how to use europepmc_api

In [82]:
from bh24_literature_mining.europepmc_api import EuropePMCClient, write_tool_mentions_to_file
from bh24_literature_mining.europepmc_api import identify_tool_mentions_in_sentences

client = EuropePMCClient()

for tool in unique_biotools[:10]:
    # Call bio.tools query and get a list of Article objects
    
    biotools_articles = client.search_mentions(tool.name)

    if len(biotools_articles) == 0:
        continue
    first_article = biotools_articles[0]

    relevant_parahraphs = client.get_relevant_paragraphs(first_article.pmcid, tool.name)
    if len(relevant_parahraphs) == 0:
        print("No relevant paragraphs found", tool.name)
        continue
    
    tool.get_topics()
    
    file_path = "/Users/vedran/Desktop/tmp.txt"
    result = identify_tool_mentions_in_sentences(first_article.pmcid, tool, relevant_parahraphs)
    write_tool_mentions_to_file(result, file_path)



No relevant paragraphs found iPHoP
No relevant paragraphs found SLiM
No relevant paragraphs found BCFtools


In [ ]:
final_data = []

for pmcid in tqdm(unique_pmcids, desc="Processing PMCIDs"):
    partial_sentences = annotations_df[annotations_df['pmc_id'] == pmcid]['partial_sentence'].tolist()
    segmented_sentences = get_full_text_xml_paragraphs(pmcid, partial_sentences)
    processed_data = process_pmcid(annotations_df, pmcid, segmented_sentences)   
    if processed_data:
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

In [ ]:
# Call cites query with a specific PubMed ID and get a list of Article objects
cites_articles = client.search_cites("32109013")
print("The count of the cites articles (within the first 3 pages):", len(cites_articles))
